In [ ]:
import pickle
import numpy as np
import pandas as pd
import glob 
import gc
import os

from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt 
import matplotlib

from numpy.random import randint

from scipy.stats import linregress 

from sklearn.impute import KNNImputer, MissingIndicator
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
path = Path('/home/jovyan/workspace/amex-challenge/archive')

In [3]:
class DateTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X["S_2"] = pd.to_datetime(X["S_2"]).astype(int)/ 10**9
        return X
    
class CoalesceTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.sort_values('S_2').groupby("customer_ID").tail(1)
        return X

In [4]:
train_data = pd.read_feather(path / f'data/train_data.ftr')
train_labels = pd.read_feather(path / f'data/train_labels.ftr')
train_data = train_data.set_index("customer_ID")
train_labels = train_labels.set_index("customer_ID")
joined = train_data.join(train_labels)


In [5]:
categorical_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
numerical_cols = set(joined.select_dtypes(include=(np.number)).columns)
non_numeric_cols = set(joined.columns).difference(numerical_cols)
cols_with_null = set(joined.columns[joined.isna().any()].tolist())

In [6]:
non_leaking_pipe = Pipeline([
    ('coalesce', CoalesceTransformer()),
    ('date', DateTransformer()),
])

In [7]:
joined_processed = non_leaking_pipe.fit_transform(joined)

In [8]:
y = joined_processed.target
X = joined_processed.drop("target", axis=1)

In [9]:
handle_null_numerical = FeatureUnion([
         ('features', KNNImputer(n_neighbors=3)),
         ('indicators', MissingIndicator())
])

preprocessor = ColumnTransformer([
    ('categorical_encoder', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('numerical_imputer', handle_null_numerical, list(cols_with_null - non_numeric_cols))
], 
remainder="passthrough")

In [10]:
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', GradientBoostingClassifier(verbose=True))
])

In [11]:
cv = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
scores = cross_validate(pipe, X, y, cv=cv, n_jobs=5,
                        scoring=('precision', 'recall', 'f1', 'accuracy', 'balanced_accuracy'),
                        return_train_score=True)

KeyboardInterrupt: 

In [ ]:
scores